In [1]:
import argparse
import logging
import sys
import numpy as np
import torch
from utils import compute_mean_dice
import pandas as pd
import os
from glob import glob
from matplotlib import pyplot as plt
import monai
import torchinfo
import nibabel as nib
from monai.transforms import AsDiscrete
from miseval import evaluate
from scipy.io import savemat
import itk
import SimpleITK as sitk
import random
import subprocess

In [2]:
atlas_mri_file = os.path.join("dataset2", "Atlas", "Identity_Feminad_Template.nii.gz")
atlas_mask_file = os.path.join("dataset2", "Atlas", "Identity_Feminad_Template_Mask.nii.gz")
fakedata_mris = os.path.join("dataset2", "Fakedata", "MRI_N4_Resample_Norm_Identity_Affine")
#os.mkdir(fakedata_mris)
fakedata_masks = os.path.join("dataset2", "Fakedata", "Mask_Resample_Identity_Affine")
#os.mkdir(fakedata_masks)

#original_masks = sorted(glob(os.path.join("dataset2", "Feminad", "Mask_Resample_Identity", "*.nii.gz")))
#affine_masks = sorted(glob(os.path.join("dataset2", "Feminad", "Mask_Resample_Identity_Affine", "*.nii.gz")))
#ants_masks = sorted(glob(os.path.join("dataset2", "Feminad", "Mask_Resample_Identity_Deformable", "*.nii.gz")))

In [3]:
atlas_mri = nib.load(atlas_mri_file).get_fdata().reshape((1,128,128,128))
atlas_mask = nib.load(atlas_mask_file).get_fdata().reshape((1,128,128,128))
affine = nib.load(atlas_mri_file).affine
header = nib.load(atlas_mri_file).header
atlas_name = os.path.join("dataset2", "Atlas", "Identity_Feminad_Template.nii.gz")
atlas_maskname = os.path.join("dataset2", "Atlas", "Identity_Feminad_Template_Mask.nii.gz")

for i in range(50):
    print(i+1, end='\r')
    randaffine = sitk.AffineTransform(3)
    randaffine.SetCenter((-64, -64, 64))
    s = random.uniform(0.95, 1.05)
    randaffine.Scale(s)
    tx, ty, tz = random.randint(-2, 2), random.randint(-2, 2), random.randint(-2, 2)
    randaffine.Translate((tx, ty, tz))
    rx, ry, rz = random.uniform(-np.pi/90, np.pi/90), random.uniform(-np.pi/90, np.pi/90), random.uniform(-np.pi/90, np.pi/90)
    randaffine.Rotate(0, 1, rz)
    randaffine.Rotate(1, 2, rx)
    randaffine.Rotate(2, 0, ry)
    
    mat_name = os.path.join("dataset2", "Fakedata", "MRI_N4_Resample_Norm_Identity_Affine", str(i) + ".mat")
    sitk.WriteTransform(randaffine, mat_name)     
    img_name = os.path.join("dataset2", "Fakedata", "MRI_N4_Resample_Norm_Identity_Affine", str(i) + ".nii.gz")
    mask_name = os.path.join("dataset2", "Fakedata", "Mask_Resample_Identity_Affine", str(i) + "_mask.nii.gz")
    cmd = "antsApplyTransforms -d 3 -i " + str(atlas_name) + " -r " + str(atlas_name) + " -o " + str(img_name) + " -t " + str(mat_name)
    subprocess.call(cmd.split(" "))
    cmd_mask = "antsApplyTransforms -d 3 -i " + str(atlas_maskname) + " -r " + str(atlas_maskname) + " -o " + str(mask_name) + " -t " + str(mat_name) + " -n NearestNeighbor"
    subprocess.call(cmd_mask.split(" "))

In [4]:
def getLog10Jacobian_deformable_ants(warp_img):
    outdir = os.path.join('dataset2', 'Fakedata', 'Log10Jacobian')
    outname = outdir + '/JacobianDeformable_' + warp_img.split('/')[-1].split('Warp')[0] + '.nii.gz'
    print(outname)
    jacobian_command = "CreateJacobianDeterminantImage 3 " + str(warp_img) + " " + str(outname) + " 1 1"
    subprocess.call(jacobian_command.split(" "))

In [5]:
from utils import get_warp_from_mat_file
import SimpleITK as sitk
from scipy.io import loadmat

mats = sorted(glob(os.path.join("dataset2", "Fakedata", "MRI_N4_Resample_Norm_Identity_Affine", "*.mat")))
atlas_name = "dataset2/Atlas/Identity_Feminad_Template.nii.gz"
truthwarp_dir = os.path.join("dataset2", "Fakedata", "DeformableWarp")
mat_dir = os.path.join("dataset2", "Fakedata", "MRI_N4_Resample_Norm_Identity_Affine")
try:
    os.mkdir(truthwarp_dir)
except:
    pass
for i, mat in enumerate(mats):    
    print(i+1, end='\r')
    read_mat = sitk.ReadTransform(mat)
    inv_affine = read_mat.GetInverse()
    
    inv_mat = mat.split('/')[-1].split('.')[0] + '_inv.mat'
    inv_matname = os.path.join(mat_dir, inv_mat)
    sitk.WriteTransform(inv_affine, inv_matname)
    name = mat.split('/')[-1].split('.')[0] + "_InverseWarp.nii.gz"
    warp = get_warp_from_mat_file(atlas_name, inv_matname)
    warp_name = os.path.join(truthwarp_dir, name)
    nib.save(warp, warp_name)
    
    getLog10Jacobian_deformable_ants(warp_name)

    
    
    #name = mat.split('/')[-1].split('.')[0] + "_Warp.nii.gz"
    #test = np.loadtxt(mat)
    #print(test)
    #flip_RAS_LPS = np.diag([-1.0, -1.0, 1.0, 1.0])
    ## test = flip_RAS_LPS @ test @ flip_RAS_LPS
    #print(test)
    #test_affine = sitk.AffineTransform(3)
    #parameters = np.zeros(3*3+3)
    #for i in range(3):
    #    for j in range(3):
    #        parameters[i*3+j] = test[i][j]
    #for i in range(3):
    #    parameters[i+3*3] = test[i][3]
    #test_affine.SetParameters(parameters)
    #outmatname = mat.split('.')[0] + '.mat'
    #sitk.WriteTransform(test_affine, outmatname)   
    #warp = get_warp_from_mat_file(atlas_name, outmatname)
    #nib.save(warp, os.path.join(truthwarp_dir, name))

dataset2/Fakedata/Log10Jacobian/JacobianDeformable_0_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_1_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_10_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_11_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_12_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_13_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_14_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_15_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_16_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_17_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_18_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_19_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_2_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/JacobianDeformable_20_Inverse.nii.gz
dataset2/Fakedata/Log10Jacobian/Jacob